**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [ ]:
!mkdir "/mydrive/yolov3"

In [ ]:
%cd gdrive/MyDrive/yolov3
!ls

**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
!echo "gengel" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

In [ ]:
%cd data/obj/images

/content/gdrive/My Drive/yolov3/darknet/data/obj/images


In [ ]:
import glob
import os

images_list = glob.glob("*.jpg")
print(images_list)

In [ ]:
images_adress_list = ["/content/gdrive/My Drive/yolov3/darknet/data/obj/images/"+ i for i in images_list]
print(images_adress_list)

In [ ]:
%cd ..

/content/gdrive/My Drive/yolov3/darknet


In [ ]:
train_file = open("data/train.txt", "w")
train_file.write("\n".join(images_adress_list[:70]))

test_file = open("data/test.txt", "w")
test_file.write("\n".join(images_adress_list[70:]))

train_file.close()
test_file.close()

**4) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show